In [14]:
import os
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Adiciona um marcador de COVID aos dados (True ou False)
def add_covid_period(df):
    pre_covid = pd.Timestamp('2020-03-11')
    first_wave_start = pd.Timestamp('2020-03-11')
    first_wave_peak = pd.Timestamp('2020-07-29')
    second_wave_start = pd.Timestamp('2020-11-01')
    second_wave_peak = pd.Timestamp('2021-03-27')
    vaccination_start = pd.Timestamp('2021-01-17')

    def is_covid_period(date):
        return pd.Timestamp('2020-01-11') <= date < pd.Timestamp('2021-09-30')

    df['is_covid'] = df.index.map(is_covid_period)

    return df

# Prepara os dados melhorados
def prepare_improved_data(df_original):
    df = df_original.copy()
    df['mes'] = df.index.month

    df = add_covid_period(df)

    # Removendo colunas desnecessárias
    cols_to_drop = ['hora', 'dia', 'dia_semana', 'semana_ano']
    df = df.drop(columns=cols_to_drop, errors='ignore')

    return df

# Imputação linear
def linear_interpolation_imputer(df):
    df_imputed = df.interpolate(method='linear')
    return df_imputed


# Processa os arquivos tratados e calcula correlações
treated_folder = "../dados_tratados/combinado"
station_folders = [f for f in os.listdir(treated_folder) if os.path.isdir(os.path.join(treated_folder, f))]

for station_folder in station_folders:
    print(f"Análise descritiva para a estação: {station_folder}\n")

    treated_files = [f for f in os.listdir(os.path.join(treated_folder, station_folder)) if f.endswith(".csv")]
    correlacoes = []
    for treated_file in reversed(treated_files):
        treated_file_path = os.path.join(treated_folder, station_folder, treated_file)
        df = pd.read_csv(treated_file_path, parse_dates=['Data e Hora'], low_memory=False)
        df.set_index('Data e Hora', inplace=True)
        df.sort_index(inplace=True)

        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.loc['2019-01-01':'2022-01-01']

        # Aplica imputação
        df = linear_interpolation_imputer(df)
        df = prepare_improved_data(df)

        if 'PM2.5' in df.columns:
            print("-" * 100)
            print(f"Arquivo tratado: {treated_file}")

            # Correlação entre PM2.5 e outras variáveis
            pm25_corr = df.corr(method='pearson')['PM2.5'].sort_values(ascending=False)
            pm25_corr = pm25_corr[pm25_corr.notnull()]  # Remove NaN
            print("\nCorrelação entre PM2.5 e outras variáveis:")
            print(pm25_corr)
            correlacoes.append(pm25_corr)


Análise descritiva para a estação: Cascata

----------------------------------------------------------------------------------------------------
Arquivo tratado: Cascata_tratado_combinado.csv

Correlação entre PM2.5 e outras variáveis:
PM2.5                    1.000000
PM10                     0.649290
Dióxido de Nitrogênio    0.395714
Monóxido de Carbono      0.226330
Dióxido de Enxofre       0.072851
mes                      0.055548
Direção do Vento         0.019715
is_covid                 0.000522
Umidade Relativa        -0.024058
Pressão Atmosférica     -0.042452
Ozônio                  -0.092205
Temperatura             -0.111329
Radiação Solar          -0.137020
Velocidade do Vento     -0.241238
Name: PM2.5, dtype: float64
Análise descritiva para a estação: Piratininga

----------------------------------------------------------------------------------------------------
Arquivo tratado: Piratininga_tratado_combinado.csv

Correlação entre PM2.5 e outras variáveis:
PM2.5           